# Programming Assignment (Dictionary lookup and autocomplete using Tries)

## Trie
A trie is a special rooted tree. that allows us to store a set of words by sharing their common prefixes.



### Trie Data Structure: Specification

A trie (or a prefix tree) data structure allows us to store a set of strings from a dictionary along with a frequency count for each strings. The following operations are supported:

|Operation|Description|Return Type|
|----|----|----| 
| `addWord(w)`	|Insert a word w into the trie	| None  | 
| `lookupWord(w)` |Return the frequency of w in the trie (0 if not present) |	int  | 
| `autoComplete(w)` |Return a list of words in the trie along with their frequencies so that `w` is a prefix of each word in the list  |	list of `(str,int)` |



## Implementation

In [1]:
import sys

class MyTrieNode:

    def __init__(self, isRootNode):
        self.isRoot = isRootNode 
        self.isWordEnd = False 
        self.count = 0 
        self.next = {} # Dictionary mapping each character from a-z to 
                       
    def addWord(self,w):
        """
        Adds word if new word to Trie, otherwise just increase count  
        """
        assert(len(w) > 0)
        
        # Child Map for current letter 
        trie_layer = self.next 
        
        # Traverse every letter of word 
        for i in range(len(w)):
            letter = w[i] 
            # If letter not in trie_layer, create new layer and traverse to next layer
            if  letter not in trie_layer:
                child_trie_layer = MyTrieNode(False)   
                trie_layer[letter]  = child_trie_layer  
                trie_layer = child_trie_layer.next 
                # Initializing new word at end of word
                if i == len(w)-1:
                    child_trie_layer.isWordEnd =  True
                    child_trie_layer.count = 1
            
            # If letter in trie_layer traverse and increase count
            else:
                layer_meta = trie_layer[letter] 
                trie_layer = trie_layer[letter].next 
                if i == len(w)-1:
                    layer_meta.isWordEnd = True # updating for shorter word
                    layer_meta.count += 1 
        return

    
    def lookupWord(self,w):
        """
        Return frequency of the word (w) in trie
        """
        assert(len(w) > 0)
        trie_layer = self.next 
        
        # Traverse every letter of word 
        for i in range(len(w)):
            letter = w[i] 
            # If letter in trie_layer, store node for count info
            if  letter in trie_layer:
                meta_layer, trie_layer = trie_layer[letter],trie_layer[letter].next                
                if i == len(w) - 1:
                    return meta_layer.count
            # Exit loop, letter not contained
            else:
                return 0
            

    def autoCompleteHelper(self,node, w,word_options):
        """
        Recursively search all paths given a prefix and return all possible words
        """
        if node == None:
            return
        
        # Add to list of reccomendations if we have found a word
        if node.isWordEnd == True:
            word_options.append((w, node.count))
        
        # Recursively traverse through all possible paths 
        for letter, following_letters in node.next.items():
            self.autoCompleteHelper(node.next[letter], w + letter, word_options)

            
            
    def autoComplete(self,w):
        """
        Build up prefix from input word, by each letter; to find all possible words with common prefix
        Returns list of possible words and their frequencies
        """
        trie_cursor = self
        trie_layer = self.next
        base_word =  ""
        no_suggestion = False
        
        for i in range(len(w)):
            letter = w[i]
            if letter not in trie_layer: # check if Prefix (letter) exists in Trie
                no_suggestion = True
                break
                
            base_word = base_word + letter 
            trie_cursor = trie_layer[letter]
            trie_layer = trie_layer[letter].next

        # Prefix does not exist in Trie
        if no_suggestion == True:
            return -1

        # Prefix exists, but end of path in trie
        elif (trie_cursor.isWordEnd == True) and (trie_layer == False):
            return 0
        
        # Pass Prefix to helper and recurse for all paths
        word_options = []
        self.autoCompleteHelper(trie_cursor, base_word,word_options)
        return word_options

## Test Cases

In [11]:
# Create a root Trie node
t= MyTrieNode(True) 

# Insert the words in lst1
lst1=['testing','test','test','test','test','testament','testing','ping','pin','pink','pine','pint','pinetree']
for w in lst1:
    t.addWord(w)

# Perform lookups
j = t.lookupWord('testy') # should return 0
print(j)
j2 = t.lookupWord('telltale') # should return 0
print(j2)
j3 = t.lookupWord ('testing') # should return 2
print(j3)

# Run autocompletes
lst3 = t.autoComplete('pi')
print('Completions for \"pi\" are : ')
print(lst3)

lst4 = t.autoComplete('tes')
print('Completions for \"tes\" are : ')
print(lst4)

0
0
2
Completions for "pi" are : 
[('pin', 1), ('ping', 1), ('pink', 1), ('pine', 1), ('pinetree', 1), ('pint', 1)]
Completions for "tes" are : 
[('test', 4), ('testing', 2), ('testament', 1)]


## Additional Test Cases

In [5]:
import sys

def lookupTest(t, lstToLookup):
    retValue = True
    for (w,k) in lstToLookup:
        j = t.lookupWord(w)
        if (j != k):
            print('\t Lookup for word %s failed. Expected result: %d, obtained from your trie: %d \n'%(w,k,j))
            retValue = False
    return retValue

def autoCompleteTest(t, stem, expResult):
    lst1 = sorted(t.autoComplete(stem))
    lstRet = sorted(expResult)
    if (len(lst1) != len(lstRet)):
        print('\t autoComplete(\"%s\") failed'%(stem))
        print('\t\t Expected: ',lstRet)
        print('\t\t Got: ', lst1)
        return False
    n = len(lstRet)
    for i in range(0,n):
        (expI,freqI) = lstRet[i]
        (gotI,freqJ) = lst1[i]
        if (expI != gotI or freqI != freqJ):
            print('autoComplete(\"%s\") failed'%(stem))
            print('\t Expected: ',lstRet)
            print('\t Got: ', lst1 )
            return False   
    return True

def runTest(specs):
    try:
        t = MyTrieNode(True)
        lNum = 0
        result = True
        spec_lines = specs.split('\n')
        for line in spec_lines:
            lNum += 1
            lst = [x.strip() for x in line.split(',')]
            if (lst[0] == 'W'):
                print('\t Insert:',lst[1])
                t.addWord(lst[1])
            elif (lst[0] == 'L'):
                print('\t Lookup:', lst[1])
                j = t.lookupWord(lst[1])
                if (j != int(lst[2])):
                    print('\t\t Failed --> expected : %s, got %d'%(lst[2],j))
                    result=False
            elif (lst[0] == 'A'):
                wrd = lst[1]
                rList = lst[2::]
                rWords = rList[0::2]
                print('\t Autocomplete: ',lst[1])
                rNums = [int(x) for x in rList[1::2] ]
                retList = sorted(zip (rWords,rNums))
#                 print(t,wrd,retList)
                result = (autoCompleteTest(t,wrd, retList)) and result
            else:
                print('Error in test specification line number %d -- Unknown command spec %s'%(lNum,lst[0]))
                sys.exit(1)
        return result
    except IOError:
        print('Unable to open',fileName)



In [6]:
str_spec1='''W,hello
W,hello
W,he
W,jello
W,jelly
L,hello,2
L,hell,0
L,howdy,0
A,he,hello,2,he,1
A,je,jello,1,jelly,1'''

rslt = runTest(str_spec1)
if (rslt):
    print('Test 1 PASSED')
else:
    print('Test 1 FAILED')

	 Insert: hello
	 Insert: hello
	 Insert: he
	 Insert: jello
	 Insert: jelly
	 Lookup: hello
	 Lookup: hell
	 Lookup: howdy
	 Autocomplete:  he
	 Autocomplete:  je
Test 1 PASSED


In [7]:
str_spec2 = '''W,hello
W,how
W,are
W,you
W,hell
L,howdy,0
W,howdy
W,arid
W,arachnid
L,orange,0
L,hello,1
L,hell,1
L,howdy,1
A,ho,howdy,1,how,1'''

rslt = runTest(str_spec2)
if (rslt):
    print('Test 2 PASSED')
else:
    print('Test 2 FAILED')

	 Insert: hello
	 Insert: how
	 Insert: are
	 Insert: you
	 Insert: hell
	 Lookup: howdy
	 Insert: howdy
	 Insert: arid
	 Insert: arachnid
	 Lookup: orange
	 Lookup: hello
	 Lookup: hell
	 Lookup: howdy
	 Autocomplete:  ho
Test 2 PASSED


In [8]:
str_spec3='''L,lolo,0
L,popo,0
W,cat
W,hat
W,mat
W,hatter
W,mad
W,maddening
W,catherine
A,cath,catherine,1'''

rslt = runTest(str_spec3)
if (rslt):
    print('Test 3 PASSED')
else:
    print('Test 3 FAILED')


	 Lookup: lolo
	 Lookup: popo
	 Insert: cat
	 Insert: hat
	 Insert: mat
	 Insert: hatter
	 Insert: mad
	 Insert: maddening
	 Insert: catherine
	 Autocomplete:  cath
Test 3 PASSED


In [9]:
str_spec4 = '''W,jelly
W,jello
W,just
W,justin
W,jejunum
W,jellyfish
L,jell,0
L,jel,0
L,jellyfish,1
L,justtin,0
L,jus,0
L,justin,1'''

rslt = runTest(str_spec4)
if (rslt):
    print('Test 4 PASSED')
else:
    print('Test 4 FAILED')



	 Insert: jelly
	 Insert: jello
	 Insert: just
	 Insert: justin
	 Insert: jejunum
	 Insert: jellyfish
	 Lookup: jell
	 Lookup: jel
	 Lookup: jellyfish
	 Lookup: justtin
	 Lookup: jus
	 Lookup: justin
Test 4 PASSED
